In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

from src.template_loader import *
from src.customization import *
from src.runner import *
from src.exporter import *

experiment_queue = []

topology_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + os.listdir("topologies"),
    description='Topologies:'
)

workload_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + os.listdir("workload_traces"),
    description='Workloads:'
)

failure_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + os.listdir("failure_traces"),
    description='Failures:'
)

# carbon_selector = widgets.SelectMultiple(
#     options=['[Keep original]'] + os.listdir("carbon_traces"),
#     description='Carbon:'
# )


name_selector = widgets.Text( value='custom_experiment.json', placeholder='Enter experiment name', description='Name:', disabled=False)

remove_selector = widgets.Dropdown(options=[], description='Remove From Queue:')
remove_button = widgets.Button(description="Remove Selected From The Queue")
remove_row = widgets.HBox([remove_selector, remove_button])

# topology_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("topologies"), description='Topology:')
topology_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Topology')
topology_row = widgets.HBox([topology_selector, topology_upload])

# workload_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("workload_traces"), description='Workload:')
workload_upload = widgets.FileUpload(multiple=False, description='Upload Workload')
workload_row = widgets.HBox([workload_selector, workload_upload])

# failure_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("failure_traces"), description='Failures:')
failure_upload = widgets.FileUpload(accept='.parquet', multiple=False, description='Upload Failures')
failure_row = widgets.HBox([failure_selector, failure_upload]) 

# carbon_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("carbon_traces"), description='Carbon:')
# carbon_upload = widgets.FileUpload(accept='.parquet', multiple=False, description='Upload Carbon Traces')
# carbon_row = widgets.HBox([carbon_selector, carbon_upload]) 

experiment_selector = widgets.Dropdown(options=os.listdir("experiments"), description='Experiment:')
experiment_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Experiment')
experiment_row = widgets.HBox([experiment_selector, experiment_upload])

add_experiment_button = widgets.Button(description="Add Experiment")
run_all_button = widgets.Button(description="Run All Experiments")
export_button = widgets.Button(description="Export the capsule as ZIP")
button_row = widgets.HBox([add_experiment_button, run_all_button, export_button])

output = widgets.Output()

def on_add_clicked(b): 
    with output:
        output.clear_output()

        if experiment_upload.value:
            experiment_file = save_uploaded_file(experiment_upload, "experiments")[0]
            refresh_dropdown(experiment_selector, "experiments", experiment_file, False)
        else:
            experiment_file = experiment_selector.value

        experiment_path = os.path.join("experiments", experiment_file)
        experiment = load_experiment_template(experiment_path)
        if experiment is None:
            return


        if topology_upload.value:
            topology_file = save_uploaded_file(topology_upload, "topologies")
            # refresh_dropdown(topology_selector, "topologies", topology_file, True)
            topology_values = topology_file
        else:
            topology_values = topology_selector.value


        if workload_upload.value:
            workload_file = save_uploaded_file(workload_upload, "workload_traces")
            # refresh_dropdown(workload_selector, "workload_traces", workload_file, True)
            workload_values = workload_file
        else:
            workload_values = workload_selector.value  #TO-DO FIX WORKLOAD FORMAT UPLOAD

        if failure_upload.value:
            failure_file = save_uploaded_file(failure_upload, "failure_traces")
            # refresh_dropdown(failure_selector, "failure_traces", failure_file, True)
            failure_values = failure_file
        else:
            failure_values = failure_selector.value


        # if carbon_upload.value:
        #     carbon_file = save_uploaded_file(carbon_upload, "carbon_traces")
        #     # refresh_dropdown(carbon_selector, "carbon_traces", carbon_file, True)
        #     carbon_values = carbon_file
        # else:
        #     carbon_values = carbon_selector.value

        experiment_name = name_selector.value

        print(f"Name: {experiment_name}")

        def clean_selection(selections):
            filtered = [s for s in selections if s != '[Keep original]']
            return filtered if filtered else None


        selections = {
            "name": experiment_name,
            "topology": clean_selection(topology_values),
            "workload": clean_selection(workload_values),
            "failures": clean_selection(failure_values),
            # "carbon": clean_selection(carbon_values)
        }
        

        updated_experiment = update_experiment_fields(experiment, selections)

        filename = experiment_name
        save_json_file(updated_experiment, filename, "experiments")
        
        experiment_queue.append(selections)
        remove_selector.options = [exp["name"] for exp in experiment_queue]
        print(f"Added experiment '{experiment_name}' to the queue. Total queued: {len(experiment_queue)}")


def on_run_all_clicked(b):

    with output:
        output.clear_output()
        run_all_experiments(experiment_queue)

def on_remove_clicked(b):
    with output:
        output.clear_output()
        to_remove = remove_selector.value
        if to_remove:
            removed = False
            for i, exp in enumerate(experiment_queue):
                if exp["name"] == to_remove:
                    del experiment_queue[i]
                    removed = True
                    break
            if removed:
                print(f"Removed experiment {to_remove} from queue.")
                remove_selector.options = [exp["name"] for exp in experiment_queue]
            else:
                print(f"Experiment {to_remove} not found in queue.")
        else:
            print("No experiment selected to remove.")

def on_export_called(b):
    create_reproducibility_zip(experiment_queue)
    print("Capsule created")

add_experiment_button.on_click(on_add_clicked)
run_all_button.on_click(on_run_all_clicked)
remove_button.on_click(on_remove_clicked)
export_button.on_click(on_export_called)

display(
    widgets.VBox([
        widgets.HTML("<b>OpenDC Experiment Configurator</b>"),
        name_selector,
        experiment_row,
        topology_row,
        workload_row,
        failure_row,
        # carbon_row,
        button_row,
        remove_row,
        output
    ])
)



# Generate readme explaining what is where
# Ask about carbon and why my new runner still fails


# Customizability

In [2]:
#TO-DO make it possible to change one or multiple parameters from each file
#TO-DO make it generate files based on provided range
#TO-DO fix output file naming
#TO-DO 

# Exporting the capsule